In [ ]:
import pyodbc
import pandas as pd
cnxn_string = r'Driver={SQL Server};Server=omop.uci.edu;Database=cords;Trusted_Connection=yes;'

In [ ]:
cnxn = pyodbc.connect(cnxn_string)
conditions = pd.DataFrame(pd.read_sql_query("""
SET NOCOUNT ON
drop table if exists #non_billable_to_billable
create table #non_billable_to_billable (
 non_billable VARCHAR(10),
 billable VARCHAR(10)
);
INSERT INTO #non_billable_to_billable (non_billable, billable)
VALUES ('173.3', '173.30')
, ('173.4', '173.40')
, ('173.5', '173.50')
, ('173.9', '173.90')
, ('174', '174.9')
, ('275.0', '275.09')
, ('278', '278.00')
, ('278.0', '278.00')
, ('284.1', '284.11')
, ('286.5', '286.52')
, ('287.4', '287.41')
, ('305.6', '305.60')
, ('366', '366.9')
, ('372.0', '372.00')
, ('401', '401.9')
, ('425.1', '425.11')
, ('466', '466.0')
, ('512.8', '512.81')
, ('516.3', '516.30')
, ('518.5', '518.51')
, ('593.7', '593.70')
, ('599.7', '599.70')
, ('704.0', '704.00')
, ('706', '706.9')
, ('733.8', '733.81')
, ('743.3', '743.30')
, ('747.3', '747.31')
, ('787.2', '787.20')
, ('793.1', '793.11')
, ('795.5', '795.51')
, ('959.1', '959.11')
, ('V12.2', 'V12.29')
, ('V13.8', 'V13.89')
, ('V19.1', 'V19.19')
, ('V25.1', 'V25.11')
, ('V61.0', 'V61.09')
,('276.6', '276.61')
,('558.4', '558.41')
,('596.8', '596.81');

drop table if exists #snomed_to_icd10cm
create table #snomed_to_icd10cm (
snomed_code VARCHAR(20),
icd10cm VARCHAR(10));
INSERT INTO #snomed_to_icd10cm (snomed_code, icd10cm)
VALUES ('77386006', 'Z33.1'),
('840539006', 'U07.1'),
('66190008', 'T07.XXXA');

with covid_positive_patients as (
    select m.person_id, min(m.measurement_date) as positive_covid_test_date from cords.dbo.measurement m, cords.dbo.concept c
    where m.measurement_concept_id = c.concept_id
    and vocabulary_id='LOINC'
    and concept_code in ('94500-6', '94309-2', '94531-1', '94500-6', '94310-0', '94533-7', '94306-8') and m.value_as_concept_id=9191
    group by person_id
), 
conditions as (
	SELECT DISTINCT a.[person_id], positive_covid_test_date,[condition_concept_id], [condition_type_concept_id],[condition_source_concept_id], [condition_start_date], [condition_end_date]
	from covid_positive_patients a
	inner join (
		select * FROM [CORDS].[dbo].[condition_occurrence] 
		where condition_concept_id!=0 OR condition_source_concept_id!=0) b
		on a.person_id=b.person_id
), 
with_condition_type as (
	select ac.person_id,positive_covid_test_date, ac.condition_concept_id, c.concept_name as condition_type, condition_source_concept_id, condition_start_date, condition_end_date from conditions ac
	left join (select * from cords.dbo.concept ) as c
	on c.concept_id=ac.condition_type_concept_id
), 
with_condition_name as (
	select ac.person_id,positive_covid_test_date, condition_concept_id, c.concept_name as snomed_name, c.domain_id as snomed_domain, c.concept_code as snomed_code, ac.condition_source_concept_id, vocabulary_id as condition_name_vocab, ac.condition_type, condition_start_date, condition_end_date from with_condition_type ac
	left join (select * from cords.dbo.concept ) as c
	on c.concept_id=ac.condition_concept_id
),
with_condition_source as (
	select person_id, positive_covid_test_date, snomed_name, snomed_domain, condition_concept_id as snomed_concept_id, snomed_code, condition_source_concept_id as source_concept_id, c.concept_code as source_code, c.vocabulary_id as source_vocab, condition_type, condition_start_date, condition_end_date from with_condition_name cn
	left join (select * from cords.dbo.concept) as c
	on c.concept_id=cn.condition_source_concept_id
),
with_source_name as (
	select person_id,positive_covid_test_date, snomed_name, snomed_concept_id, snomed_code, c.concept_name as source_name, c.domain_id as source_domain, source_concept_id, source_code, source_vocab, condition_type, condition_start_date, condition_end_date from with_condition_source cs
	left join (select * from cords.dbo.concept) as c
	on cs.source_vocab=c.vocabulary_id and
	c.concept_code=cs.source_code
	where domain_id = 'condition'
),
conditions_after_diagnosis_and_year_prior as (
select person_id, snomed_name, snomed_concept_id, snomed_code, source_name, source_domain, source_concept_id, source_code, 
	 source_vocab, condition_type, condition_start_date, condition_end_date from with_source_name
where condition_start_date >= dateadd(year, -1, positive_covid_test_date) 
and condition_start_date <= dateadd(day, 1, positive_covid_test_date) 
),
with_billable_ICD9_conditions as (
	select * from conditions_after_diagnosis_and_year_prior where source_vocab != 'ICD9CM'
	UNION
	select a.person_id, snomed_name, snomed_concept_id, snomed_code, source_name, source_domain, source_concept_id,
	CASE
		WHEN non_billable is NOT NULL THEN replace(billable, '.', '') 
		ELSE replace(source_code, '.', '') 
	END as source_code, source_vocab, condition_type, condition_start_date, condition_end_date
	from conditions_after_diagnosis_and_year_prior a
	left join (
		select * from #non_billable_to_billable
	) as b
	on a.source_code=b.non_billable
	where a.source_vocab='ICD9CM'
),
with_icd9cm_to_icd10cm as (
	select * from with_billable_ICD9_conditions where source_vocab != 'ICD9CM'
	UNION
	select person_id, snomed_name, snomed_concept_id, snomed_code, source_name, source_domain, source_concept_id, b.icd10cm as source_code, 
	'ICD10CM' as source_vocab, condition_type, condition_start_date, condition_end_date
	from with_billable_ICD9_conditions a
	left join (
		select icd9cm, icd10cm from (
			select *, ROW_NUMBER() OVER (PARTITION BY icd9cm order by icd10cm asc) as rw from CBMI.dbo.icd9toicd10cmgem) b
			where b.rw=1 
	) as b
	on a.source_code = b.icd9cm
	where source_vocab='ICD9CM'
), 
with_snomed_to_icd10cm as (
	select person_id, snomed_name, snomed_concept_id, snomed_code, source_name, source_domain, source_concept_id, replace(source_code, '.', '') as source_code, 
		source_vocab, condition_type, condition_start_date, condition_end_date from with_icd9cm_to_icd10cm where source_vocab != 'SNOMED'
	UNION
	select person_id, snomed_name, snomed_concept_id, a.snomed_code, source_name, source_domain, source_concept_id, replace(b.icd10cm, '.','') as source_code, 
		'ICD10CM' as source_vocab, condition_type, condition_start_date, condition_end_date from with_icd9cm_to_icd10cm a
	left join (	select * from #snomed_to_icd10cm) as b
	on a.source_code=b.snomed_code
	where source_vocab='SNOMED'
),
with_icd10cm_names as (
	select person_id, snomed_name, snomed_concept_id, snomed_code, source_name, source_domain, source_concept_id, source_code, source_vocab, b.concept_name as icd10cm_name, condition_type, condition_start_date, condition_end_date
	from with_snomed_to_icd10cm a
	left join (
	select * from cords.dbo.concept where vocabulary_id='ICD10CM'
	) b on a.source_code=replace(b.concept_code, '.', '')
),
merge_dates as (
	select person_id, icd10cm_name, source_code as icd10cm_code, condition_type,  min([condition_start_date]) as condition_start_date, CASE WHEN max([condition_end_date]) IS NULL THEN max(condition_start_date) ELSE max([condition_end_date]) END AS condition_end_date
	from with_icd10cm_names
	group by  person_id, icd10cm_name, source_code, condition_type
),
merge_types as (
	select person_id, icd10cm_name, icd10cm_code, string_agg(condition_type, ', ') within group(order by condition_start_date asc, condition_end_date asc) as condition_types,  min([condition_start_date]) as condition_start_date, max([condition_end_date]) AS condition_end_date
	from merge_dates
	group by person_id, icd10cm_name, icd10cm_code
),

n_recent_conditions AS ( 
	select FORMAT(RowNumber, '000') as RowNumber, person_id, icd10cm_name, icd10cm_code, condition_types, condition_start_date, condition_end_date from (
		SELECT ROW_NUMBER() OVER (PARTITION BY person_id ORDER BY condition_start_date asc, condition_end_date asc) as RowNumber, * 
		FROM merge_types 
	) as a
	--where a.RowNumber <= 55
)
select RowNumber, person_id,  icd10cm_name, replace(icd10cm_code, '.', '') as icd10cm_code, condition_types, condition_start_date, condition_end_date from n_recent_conditions
order by person_id, RowNumber
""", cnxn))
cnxn.close()

In [ ]:
#Reshape conditions from long to wide format
tmp = []
variables =  ['icd10cm_name', 'icd10cm_code', 'condition_types', 'condition_start_date', 'condition_end_date']
max_row_num = conditions['RowNumber'].max()
ordered_column_names = [v + "_" + "%03d"%i for i in range(1, int(max_row_num) + 1) for v in variables]
conditions.fillna(value="None", inplace=True)
conditions[['person_id','icd10cm_code']] = conditions[['person_id','icd10cm_code']].astype(str)
for var in variables:
    conditions['tmp_idx'] = var + '_' + conditions.RowNumber.astype(str)
    tmp.append(conditions.pivot(index='person_id', columns='tmp_idx', values=var))
    
reshape = pd.concat(tmp, axis=1)
conditions_wide = reshape[ordered_column_names]
conditions_wide.reset_index(inplace=True)
conditions_wide.head()

In [ ]:
#conditions_wide = conditions_wide.astype(str)

In [ ]:
conditions_wide.to_csv("conditions.csv", sep='\t', index=False)

In [ ]:
conditions_wide

In [ ]:
conditions_wide[conditions_wide['condition_start_date_250'].astype(str) != "nan"]

In [ ]:
duplicates = conditions_wide['person_id'].duplicated()

In [ ]:
duplicates.value_counts()